In [36]:
%matplotlib inline 
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import metrics
from sklearn import linear_model
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [60]:
df = pd.read_csv('data/kc_house_data.csv')

df

id        date     price  bedrooms  bathrooms  sqft_living  \
0      7129300520  10/13/2014  221900.0         3       1.00         1180   
1      6414100192   12/9/2014  538000.0         3       2.25         2570   
2      5631500400   2/25/2015  180000.0         2       1.00          770   
3      2487200875   12/9/2014  604000.0         4       3.00         1960   
4      1954400510   2/18/2015  510000.0         3       2.00         1680   
...           ...         ...       ...       ...        ...          ...   
21592   263000018   5/21/2014  360000.0         3       2.50         1530   
21593  6600060120   2/23/2015  400000.0         4       2.50         2310   
21594  1523300141   6/23/2014  402101.0         2       0.75         1020   
21595   291310100   1/16/2015  400000.0         3       2.50         1600   
21596  1523300157  10/15/2014  325000.0         2       0.75         1020   

       sqft_lot  floors waterfront  view  ...          grade sqft_above  \
0          5650     1.0        NaN  NONE  ...      7 Average       1180   
1          7242     2.0         NO  NONE  ...      7 Average       2170   
2         10000     1.0         NO  NONE  ...  6 Low Average        770   
3          5000     1.0         NO  NONE  ...      7 Average       1050   
4          8080     1.0         NO  NONE  ...         8 Good       1680   
...         ...     ...        ...   ...  ...            ...        ...   
21592      1131     3.0         NO  NONE  ...         8 Good       1530   
21593      5813     2.0         NO  NONE  ...         8 Good       2310   
21594      1350     2.0         NO  NONE  ...      7 Average       1020   
21595      2388     2.0        NaN  NONE  ...         8 Good       1600   
21596      1076     2.0         NO  NONE  ...      7 Average       1020   

       sqft_basement yr_built  yr_renovated  zipcode      lat     long  \
0                0.0     1955           0.0    98178  47.5112 -122.257   
1              400.0     1951        1991.0    98125  47.7210 -122.319   
2                0.0     1933           NaN    98028  47.7379 -122.233   
3              910.0     1965           0.0    98136  47.5208 -122.393   
4                0.0     1987           0.0    98074  47.6168 -122.045   
...              ...      ...           ...      ...      ...      ...   
21592            0.0     2009           0.0    98103  47.6993 -122.346   
21593            0.0     2014           0.0    98146  47.5107 -122.362   
21594            0.0     2009           0.0    98144  47.5944 -122.299   
21595            0.0     2004           0.0    98027  47.5345 -122.069   
21596            0.0     2008           0.0    98144  47.5941 -122.299   

       sqft_living15  sqft_lot15  
0               1340        5650  
1               1690        7639  
2               2720        8062  
3               1360        5000  
4               1800        7503  
...              ...         ...  
21592           1530        1509  
21593           1830        7200  
21594           1020        2007  
21595           1410        1287  
21596           1020        1357  

[21597 rows x 21 columns]

In [62]:
df.dropna(inplace=True)

In [65]:
df.replace({'NO': 0, 'YES': 1}, inplace = True)

In [77]:
df['view'].unique()

array(['NONE', 'GOOD', 'EXCELLENT', 'AVERAGE', 'FAIR'], dtype=object)

In [78]:
df.replace({'NONE': 0, 'FAIR': 1, 'AVERAGE': 2, 'GOOD': 3, 'EXCELLENT': 4}, inplace= True)

In [79]:
df['yr_renovated'].mask(df['yr_renovated'] == 0, df['yr_built'], inplace = True)


In [80]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [99]:
numeric_df = df.drop(columns = ['id', 'date', 'sqft_lot15', 'sqft_living15', 'sqft_lot'])

In [100]:
numeric_df['grade'].unique()

array(['7 Average', '8 Good', '11 Excellent', '9 Better', '6 Low Average',
       '5 Fair', '10 Very Good', '12 Luxury', '4 Low', '3 Poor',
       '13 Mansion'], dtype=object)

In [101]:
numeric_df.replace({'7 Average': 7, '8 Good': 8, '11 Excellent': 11, '9 Better': 9, '6 Low Average': 6, '5 Fair': 5, '10 Very Good': 10, '12 Luxury': 12, '4 Low': 4, '3 Poor': 3, '13 Mansion': 13}, inplace = True)

In [102]:
numeric_df['condition'].unique()

array(['Average', 'Very Good', 'Good', 'Poor', 'Fair'], dtype=object)

In [103]:
numeric_df.replace({'Poor': 0, 'Fair': 1, 'Average': 2, 'Good': 3, 'Very Good': 4}, inplace = True)

In [104]:
numeric_df.drop(columns = 'sqft_basement', inplace = True)

In [105]:
numeric_df.corr()

price  bedrooms  bathrooms  sqft_living    floors  \
price         1.000000  0.305489   0.526155     0.706189  0.259505   
bedrooms      0.305489  1.000000   0.512243     0.573575  0.180485   
bathrooms     0.526155  0.512243   1.000000     0.753846  0.505187   
sqft_living   0.706189  0.573575   0.753846     1.000000  0.359407   
floors        0.259505  0.180485   0.505187     0.359407  1.000000   
waterfront    0.274212 -0.005833   0.065688     0.111491  0.018382   
view          0.396862  0.080577   0.180923     0.285506  0.027518   
condition     0.034367  0.020074  -0.130287    -0.062319 -0.261013   
grade         0.664146  0.354243   0.664748     0.764251  0.459843   
sqft_above    0.612014  0.474272   0.685677     0.876176  0.529101   
yr_built      0.049345  0.153229   0.504841     0.314220  0.487052   
yr_renovated  0.100023  0.163990   0.534258     0.338318  0.499460   
zipcode      -0.049502 -0.147255  -0.199625    -0.196537 -0.058130   
lat           0.306607 -0.005917   0.029930     0.058394  0.058190   
long          0.021215  0.129370   0.222755     0.239521  0.129769   

              waterfront      view  condition     grade  sqft_above  yr_built  \
price           0.274212  0.396862   0.034367  0.664146    0.612014  0.049345   
bedrooms       -0.005833  0.080577   0.020074  0.354243    0.474272  0.153229   
bathrooms       0.065688  0.180923  -0.130287  0.664748    0.685677  0.504841   
sqft_living     0.111491  0.285506  -0.062319  0.764251    0.876176  0.314220   
floors          0.018382  0.027518  -0.261013  0.459843    0.529101  0.487052   
waterfront      1.000000  0.409773   0.016454  0.083034    0.077165 -0.024068   
view            0.409773  1.000000   0.046354  0.248679    0.170726 -0.056645   
condition       0.016454  0.046354   1.000000 -0.147810   -0.157958 -0.366938   
grade           0.083034  0.248679  -0.147810  1.000000    0.758289  0.443286   
sqft_above      0.077165  0.170726  -0.157958  0.758289    1.000000  0.418850   
yr_built       -0.024068 -0.056645  -0.366938  0.443286    0.418850  1.000000   
yr_renovated   -0.001108 -0.023260  -0.401191  0.456067    0.429655  0.911762   
zipcode         0.030391  0.086479   0.001685 -0.184120   -0.257359 -0.344252   
lat            -0.015935  0.008403  -0.022250  0.117425    0.009045 -0.144089   
long           -0.042324 -0.078500  -0.105823  0.200680    0.342034  0.408623   

              yr_renovated   zipcode       lat      long  
price             0.100023 -0.049502  0.306607  0.021215  
bedrooms          0.163990 -0.147255 -0.005917  0.129370  
bathrooms         0.534258 -0.199625  0.029930  0.222755  
sqft_living       0.338318 -0.196537  0.058394  0.239521  
floors            0.499460 -0.058130  0.058190  0.129769  
waterfront       -0.001108  0.030391 -0.015935 -0.042324  
view             -0.023260  0.086479  0.008403 -0.078500  
condition        -0.401191  0.001685 -0.022250 -0.105823  
grade             0.456067 -0.184120  0.117425  0.200680  
sqft_above        0.429655 -0.257359  0.009045  0.342034  
yr_built          0.911762 -0.344252 -0.144089  0.408623  
yr_renovated      1.000000 -0.316609 -0.130761  0.380645  
zipcode          -0.316609  1.000000  0.267763 -0.562154  
lat              -0.130761  0.267763  1.000000 -0.133840  
long              0.380645 -0.562154 -0.133840  1.000000

In [106]:
import seaborn as sns
sns.heatmap(numeric_df.corr(), linewidth = 0.5);

In [107]:
# from sklearn import preprocessing

# standardized_data = preprocessing.scale(numeric_df)

# df_standard = pd.DataFrame(standardized_data, columns = numeric_df.columns)

In [113]:
# import statsmodels.api as sm

# y= df_standard['price']
# X = sm.add_constant(df_standard.drop(columns = ['price', 'floors']))

# model = sm.OLS(y, X).fit()

# model.summary()

In [114]:
import statsmodels.api as sm

y= numeric_df['price']
X = sm.add_constant(numeric_df.drop(columns = ['price', 'floors', 'condition']))

model = sm.OLS(y, X).fit()

model.summary()

/Users/markhurtado/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     3046.
Date:                Mon, 28 Mar 2022   Prob (F-statistic):               0.00
Time:                        15:03:15   Log-Likelihood:            -2.1509e+05
No. Observations:               15762   AIC:                         4.302e+05
Df Residuals:                   15749   BIC:                         4.303e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          1.13e+07   3.29e+06      3.432      0.001    4.85e+06    1.78e+07
bedrooms     -3.636e+04   2217.075    -16.399      0.000   -4.07e+04    -3.2e+04
bathrooms     4.781e+04   3740.442     12.781      0.000    4.05e+04    5.51e+04
sqft_living    158.9162      4.772     33.300      0.000     149.562     168.270
waterfront    6.116e+05   2.06e+04     29.701      0.000    5.71e+05    6.52e+05
view          5.436e+04   2529.156     21.495      0.000    4.94e+04    5.93e+04
grade         9.709e+04   2460.944     39.451      0.000    9.23e+04    1.02e+05
sqft_above      37.1527      4.563      8.142      0.000      28.209      46.097
yr_built     -3254.2952    139.370    -23.350      0.000   -3527.477   -2981.114
yr_renovated   304.4729    141.264      2.155      0.031      27.580     581.366
zipcode       -625.0006     38.573    -16.203      0.000    -700.608    -549.393
lat           6.015e+05   1.26e+04     47.627      0.000    5.77e+05    6.26e+05
long         -2.177e+05    1.5e+04    -14.486      0.000   -2.47e+05   -1.88e+05
==============================================================================
Omnibus:                    13441.460   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1379314.221
Skew:                           3.573   Prob(JB):                         0.00
Kurtosis:                      48.267   Cond. No.                     1.99e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.99e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""